# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin

import pickle

[nltk_data] Downloading package punkt to /Users/michael/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/michael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/michael/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('data/DisasterResponse.db', engine)

In [5]:
# replace the value 2 of columns related to 0
# as they are not disaster messages
# (ASK A MENTOR: https://knowledge.udacity.com/questions/954962)
df.related = df.related.replace(2, 0)

In [6]:
X = df.message.values
Y = df.iloc[:, 4:].values

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    # stop_words = stopwords.words("english") + list(string.punctuation)
    # lemmatizer = WordNetLemmatizer()
    
    # text = text.lower().strip()
    # tokens = word_tokenize(text)
    
    # tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # return tokens

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(
        class_weight='balanced',
        n_jobs = -1
        ))),])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

In [ ]:
pipeline.score(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def model_score (model):

    Y_pred = model.predict(X_test)

    list_precision, list_recall, list_f1, list_accuracy = [], [], [], []

    # Iterate 36 target columns and generate a classification report for each
    for i, col in enumerate(df.columns[4:]):
        
        rslt = classification_report(Y_test[:, i], Y_pred[:, i])
        
        # weighted avg scores are in the 2nd last line: 
        score_line = rslt.split('\n')[-2]
        score_line_split = score_line.split()
        
        # scores are in the 2nd to 4th places of splitted texts of score_line
        precision_score = float (score_line_split[2])
        list_precision.append(precision_score)
        
        recall_score = float (score_line_split[3])
        list_recall.append(recall_score)
        
        f1_score = float (score_line_split[4])
        list_f1.append(f1_score)
        
        # accuracy is the same as weighted avg recall
        # accuracy_score = float (rslt.split('\n')[-4].split()[1])
        # list_accuracy.append(accuracy_score)
        
        print(f'{i} Target column {col}:')
        print(rslt)
        print()
        
    print ('mean of weighted avg precision: {:.2f}'.format(sum(list_precision)/len(list_precision)))
    print ('mean of weighted avg recall: {:.2f}'.format(sum(list_recall)/len(list_recall)))
    print ('mean of weighted avg f1: {:.2f}'.format(sum(list_f1)/len(list_f1)))
    print ('mean of accuracy: {:.2f}'.format(sum(list_recall)/len(list_recall)))

In [ ]:
model_score (pipeline)

In [ ]:
pipeline.score(X_test, Y_test)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [100, 300, 500],
    'clf__estimator__min_samples_split': [2, 6, 10, 14, 18],
    }

# Create a grid search object using the pipeline and parameters
grid_search = GridSearchCV(pipeline, parameters, cv = 3, verbose = 3)
# grid_search = GridSearchCV(pipeline, parameters, cv = 3, verbose=3, scoring = 'accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, Y_train)

# Print the best hyperparameters found by grid search
print("Best hyperparameters:", grid_search.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
model_score (grid_search)

In [ ]:
model_score (grid_search)

In [ ]:
grid_search.score(X_test, Y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# # Improve tokenize by adding stop words
# def tokenize_improved(text):
    
#     # add stop_words
#     stop_words = stopwords.words("english") + list(string.punctuation)
    
#     # tokenize text
#     tokens = word_tokenize(text)
    
#     # initiate lemmatizer
#     lemmatizer = WordNetLemmatizer()
    
#     # iterate through each token
#     clean_tokens = []
#     for tok in tokens:
        
#         if tok not in stop_words:       
#             # lemmatize, normalize case, and remove leading/trailing white space
#             clean_tok = lemmatizer.lemmatize(tok).lower().strip()
#             clean_tokens.append(clean_tok)
#     return clean_tokens

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
pipeline_improved = Pipeline([
    ('features', FeatureUnion(
        [('text_pipeline',
          Pipeline([
              ('vect', CountVectorizer(tokenizer=tokenize)),
              ('tfidf', TfidfTransformer()),
              ])),
         ('isverb', StartingVerbExtractor())
         ])),
    ('clf', MultiOutputClassifier(
        RandomForestClassifier(
        class_weight='balanced',
        n_jobs = -1
        )))
    ])




# pipeline_improved = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize_improved)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', MultiOutputClassifier(RandomForestClassifier(
#         class_weight='balanced',
#         n_jobs = -1
#         ))),])

In [ ]:
pipeline_improved.fit(X_train, Y_train)

In [ ]:
pipeline_improved.score(X_test, Y_test)

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [100, 300, 500],
    'clf__estimator__min_samples_split': [2, 6, 10, 14, 18],
    }

# Create a grid search object using the pipeline and parameters
grid_search_improved = GridSearchCV(pipeline_improved, parameters, cv = 3, verbose = 3)
# grid_search = GridSearchCV(pipeline, parameters, cv = 3, verbose=3, scoring = 'accuracy')

# Fit the grid search to the data
grid_search_improved.fit(X_train, Y_train)

# Print the best hyperparameters found by grid search
print("Best hyperparameters_improved:", grid_search.best_params_)

In [ ]:
model_score (grid_search_improved)

In [ ]:
grid_search_improved.score(X_test, Y_test)

In [ ]:
grid_search_improved.get_params()

### 9. Export your model as a pickle file

In [ ]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(grid_search_improved, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.